In [6]:
import cv2
import numpy as np
from keras_facenet import FaceNet
import pickle
from sklearn.preprocessing import LabelEncoder

# Load the trained SVM model
with open('fix_model_facenet_160x160.pkl', 'rb') as f:
    model = pickle.load(f)

# Load the FaceNet model
embedder = FaceNet()

# Load the label encoder
encoder = LabelEncoder()
LABEL = ['abed', 'budi', 'gibran', 'iyal', 'vicky']
encoder.fit(LABEL)  # Assuming Y is still in memory or you need to reload it

# Function to preprocess the face image
def preprocess_face(face_img):
    face_img = cv2.resize(face_img, (160, 160))  # Resize to 160x160
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)
    return face_img

# Function to get the embedding of a face image
def get_embedding(face_img):
    face_img = preprocess_face(face_img)
    yhat = embedder.embeddings(face_img)
    return yhat[0]

# Function to detect faces in an image using OpenCV's Haar cascades
def detect_faces(frame, face_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Load OpenCV's pre-trained Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define threshold for confidence score
confidence_threshold = 0.5  # Adjust this threshold as needed

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces in the frame
    faces = detect_faces(frame, face_cascade)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]

        # Get the embedding for the face image
        embedding = get_embedding(face_img)

        # Reshape the embedding for prediction
        embedding = embedding.reshape(1, -1)

        # Predict the identity and confidence score
        ypred = model.predict(embedding)
        confidence_score = model.predict_proba(embedding)[0][ypred[0]]

        # Decode the label
        label = encoder.inverse_transform(ypred)[0]

        # Draw a rectangle around the face and put the label and confidence score
        if confidence_score >= confidence_threshold:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            text = f'{label} ({confidence_score:.2f})'
            cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 15ms/step


In [5]:
import os
import numpy as np
import pickle
import cv2 as cv
from sklearn.preprocessing import LabelEncoder
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from datetime import datetime
import threading

# Load the trained SVM model
with open('fix_model_facenet_160x160.pkl', 'rb') as f:
    model = pickle.load(f)

# Initialize MTCNN and FaceNet models
IMG_DETECTOR = MTCNN()
EMBEDDER = FaceNet()

# Load the label encoder
encoder = LabelEncoder()
LABEL = ['abed', 'budi', 'gibran', 'iyal', 'vicky']
encoder.fit(LABEL)  # Assuming Y is still in memory or you need to reload it

# Pre-warm the models with a dummy image
def warm_up_models():
    # Create a dummy image
    dummy_img = np.zeros((160, 160, 3), dtype=np.float32)
    dummy_img_expanded = np.expand_dims(dummy_img, axis=0)
    
    # Warm-up FaceNet
    EMBEDDER.embeddings(dummy_img_expanded)
    print("FaceNet model warmed up.")
    
    # Warm-up MTCNN
    dummy_img_uint8 = (dummy_img * 255).astype(np.uint8)  # Convert to uint8
    IMG_DETECTOR.detect_faces(dummy_img_uint8)
    print("MTCNN model warmed up.")

# Warm-up the models in a separate thread
threading.Thread(target=warm_up_models).start()

def get_prediction(embedding):
    # Perform prediction
    predict = model.predict([embedding])
    predict_proba = model.predict_proba([embedding])
    return predict, predict_proba

# Initialize the webcam
cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces in the frame
    faces = IMG_DETECTOR.detect_faces(frame)

    for face in faces:
        x, y, w, h = face['box']
        face_img = frame[y:y+h, x:x+w]

        # Preprocess the face image
        face_img = cv.resize(face_img, (160, 160))
        face_img = face_img.astype('float32')
        face_img = np.expand_dims(face_img, axis=0)

        # Get the embedding for the face image
        embedding = EMBEDDER.embeddings(face_img)[0]

        # Get prediction
        predict, predict_proba = get_prediction(embedding)

        # Decode the label
        predicted_label = encoder.inverse_transform(predict)[0]
        confidence_score = predict_proba[0][predict[0]]

        # Check confidence score and determine if the prediction should be considered unknown
        if confidence_score < 0.5:
            predicted_label = "unknown"

        # Draw a rectangle around the face and put the label
        cv.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv.putText(frame, predicted_label, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv.imshow('Face Recognition', frame)

    # Break the loop if 'q' is pressed
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv.destroyAllWindows()


1/1 [==============================] - 2s 2s/step
FaceNet model warmed up.
1/1 [==============================] - 0s 29ms/step
MTCNN model warmed up.
1/1 [==============================] - 0s 29ms/step


c:\Users\abednego\AppData\Local\Programs\Python\Python38\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
c:\Users\abednego\AppData\Local\Programs\Python\Python38\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
c:\Users\abednego\AppData\Local\Programs\Python\Python38\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
c:\Users\abednego\AppData\Local\Programs\Python\Python38\lib\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 